<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Char_RNN_Thierry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
This code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself
T Martin, 12/11/2021
"""

'\nThis code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself\nT Martin, 12/11/2021\n'

# Import base libraries

In [1]:
import numpy as np
import pandas as pd
import base64
import requests
import tensorflow.keras as keras
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# Get Text data

In [2]:
def get_text(url):
  '''
    The function get_text will load a txt file at the specified url

    Args:
        url: The url where the text is located. 

    Returns:
        req!: a clean text
  '''

  req = requests.get(url)
  req = req.text
  return req

In [3]:
# Import the big text data
url = "http://f8cho.free.fr/RABELAIS.txt"


In [4]:
text=get_text(url)

In [ ]:
shakespeare_url='https://homl.info/shakespeare'
filepath=tf.keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
  shakespeare_text=f.read()
text=shakespeare_text

# Data preprocessing

In [5]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([text]))-1

In [7]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# Transform datecaset by using windows
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length, shift=1,drop_remainder=True)
dataset=dataset.flat_map(lambda window: window.batch(window_length))
# Shuffle the dataset with a batch size
batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
# One hot encoding
dataset=dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
# prefetching
dataset=dataset.prefetch(1)

# Model definition

In [8]:
model=tf.keras.models.Sequential([
    tf.keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))    
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',run_eagerly=True)


In [9]:
model.fit(dataset,epochs=20)

Epoch 1/20
    692/Unknown - 1876s 3s/step - loss: 2.3778

KeyboardInterrupt: ignored

In [10]:
def preprocess(texts):
  X=np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X,max_id)

In [11]:
def next_char(text,temperature=1):
  X_new=preprocess([text])
  y_proba=model.predict(X_new)[0,-1:,:]
  rescaled_logits=tf.math.log(y_proba)/temperature
  char_id=tf.random.categorical(rescaled_logits, num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [12]:
def complete_text(text,n_chars=50,temperature=1):
  for _ in range(n_chars):
    text+=next_char(text,temperature)
  return text

In [15]:
result=complete_text("Pantagruel ",temperature=1)

In [16]:
print(result)

Pantagruel cetture es de commes avact de pan la qu'il nonnêqr
